In [103]:
import scipy
import numpy             as     np
import matplotlib.pyplot as     plt
import pylab             as     pl

from   scipy             import optimize
from   scipy.optimize    import approx_fprime, minimize, Bounds
from   scipy.stats       import multivariate_normal

In [104]:
def jeffreys_prior(x, sigma):
    # unnormalised.  Applied to line flux, line ratio & velocity dispersion, both positive definite.  
    return  1. / sigma

In [105]:
def gaussian_prior(x, mu, sigma):
    # normalised.  Applied to redrock z.
    return  (1. / np.sqrt(2. * np.pi) / sigma) * np.exp(-0.5 * (x - mu)**2. / sigma**2.)     

In [131]:
def prior(z, v, r, A):
    # Gaussian prior in redshift.
    muz  = 1.6 # rr_z
    sigz = 0.1 # rr_sigz

    # Jeffreys prior in line flux, dispersion, line ratio. 
    sigv = 100.
    sigr =   1.
    sigA = 100. 
    
    # unnormalised (scalar).
    return gaussian_prior(z, muz, sigz) * jeffreys_prior(v, sigv) * jeffreys_prior(r, sigr) * jeffreys_prior(A, sigA)

In [121]:
def X2(x):  
  z       = x[0]
  v       = x[1]
  r       = x[2] 
  A       = x[3]
    
  _mean   = [0, 0, 0, 0]
  _cov    = [[1, 0, 0, 0], [0, 2, 0, 0], [0, 0, 3, 0], [0, 0, 0, 5]]
    
  return -2. * np.log(multivariate_normal.pdf(x, _mean, _cov))  

In [122]:
def mlogprior(x):
    z     = x[0]
    v     = x[1]
    r     = x[2]
    A     = x[3]
    
    # prior on the parameter space.
    return -np.log(prior(z, v, r, A))

In [123]:
def mloglike(x):
    return -X2(x) / 2. 

In [124]:
def mlogpos(x):
    return mloglike(x) + mlogprior(x)

In [136]:
def gradient(x):
  return optimize.approx_fprime(x, mlogpos, eps)

In [137]:
oo = np.array([1.7, 22.5, 0.5, 50.])
x0 = np.array([1.5, 32.5, 3.5, 5.5])
x1 = np.array([2.0, 3.0, 3.5, 5.5])

In [138]:
mlogprior(oo)

8.326693812186807

In [139]:
mlogpos(x0)

-267.3038256781296

In [140]:
mlogpos(x1)

1.1336743218703695

In [141]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin_bfgs.html
result = scipy.optimize.fmin_bfgs(mlogpos, oo, fprime=gradient, gtol=1e-05, norm=-np.inf, epsilon=1.4901161193847656e-08, maxiter=None, full_output=True, disp=1, retall=1, callback=None)

         Current function value: -380.831348
         Iterations: 1
         Function evaluations: 120
         Gradient evaluations: 108


/Users/MJWilson/opt/miniconda3/envs/desihigh/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
/Users/MJWilson/opt/miniconda3/envs/desihigh/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:497: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/Users/MJWilson/opt/miniconda3/envs/desihigh/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
/Users/MJWilson/opt/miniconda3/envs/desihigh/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/Users/MJWilson/opt/miniconda3/envs/desihigh/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
/Users/MJWilson/opt/miniconda3/envs/desihigh/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:497:

In [142]:
# Bounded, no Hessian returned.
# bounds = [(-1.0, 10.), (-40., 40.)]
# result = scipy.optimize.fmin_l_bfgs_b(chi2, x0, approx_grad=True, args=(mean, cov), bounds=bounds)

In [143]:
optimize.approx_fprime(x0, mlogpos, eps)

array([-11.5       , -16.25000381,  -1.16666412,  -1.10000229])

In [144]:
len(result)

8

In [145]:
# Parameters which minimize f, i.e., f(xopt) == fopt.
# Minimum value.
# Value of gradient at minimum, f’(xopt), which should be near 0.
# Value of 1/f’’(xopt), i.e., the inverse Hessian matrix.
# Number of function_calls made.
# Number of gradient calls made.
# Maximum number of iterations exceeded. 2 : Gradient and/or function calls not changing. 3 : NaN result encountered. 
# The value of xopt at each iteration. Only returned if retall is True.
result

(array([ 1.21231973, 23.16101247,  0.50979308, 50.58756664]),
 -380.83134844434306,
 array([-39.98034668, -11.58050537,  -0.1699295 , -10.11750793]),
 array([[ 1.04328469e-02, -3.78285933e-02, -5.26078184e-04,
         -2.99292134e-02],
        [-3.78285933e-02,  2.92054879e+00,  2.84068673e-02,
          1.70214457e+00],
        [-5.26078184e-04,  2.84068673e-02,  1.00042017e+00,
          2.51763255e-02],
        [-2.99292134e-02,  1.70214457e+00,  2.51763255e-02,
          2.50856414e+00]]),
 120,
 108,
 2,
 [array([ 1.7, 22.5,  0.5, 50. ]),
  array([ 1.21231973, 23.16101247,  0.50979308, 50.58756664])])

In [146]:
# Value of 1/f’’(xopt), i.e., the inverse Hessian matrix.
ihess = result[3]

In [147]:
ihess

array([[ 1.04328469e-02, -3.78285933e-02, -5.26078184e-04,
        -2.99292134e-02],
       [-3.78285933e-02,  2.92054879e+00,  2.84068673e-02,
         1.70214457e+00],
       [-5.26078184e-04,  2.84068673e-02,  1.00042017e+00,
         2.51763255e-02],
       [-2.99292134e-02,  1.70214457e+00,  2.51763255e-02,
         2.50856414e+00]])

In [148]:
hess = np.linalg.inv(ihess)
hess

array([[ 1.00990821e+02,  1.00209878e+00,  1.14444712e-02,
         5.24830389e-01],
       [ 1.00209878e+00,  5.76368551e-01, -6.29950079e-03,
        -3.79066239e-01],
       [ 1.14444712e-02, -6.29950079e-03,  9.99906442e-01,
        -5.62424725e-03],
       [ 5.24830389e-01, -3.79066239e-01, -5.62424725e-03,
         6.62161622e-01]])

In [149]:
# https://astrostatistics.psu.edu/su11scma5/HeavensLecturesSCMAVfull.pdf
# Note:  marginal errors.
merr = np.sqrt(np.diag(ihess))
merr

array([0.10214131, 1.70896132, 1.00021006, 1.58384473])

In [153]:
# x, y = np.mgrid[-1:1:.01, -1:1:.01]
# pos  = np.dstack((x, y))
# rv   = multivariate_normal([0.5, -0.2], [[2.0, 0.3], [0.3, 0.5]])

In [154]:
# fig  = plt.figure()
# ax   = fig.add_subplot(111)
# cax  = fig.add_axes([0.27, 0.8, 0.5, 0.05])
# im   = ax.contourf(x, y, X2(pos))
# fig.colorbar(im, cax=cax, orientation='horizontal')

In [155]:
print('Done.')

Done.
